In [247]:
import sys
import os
import psycopg2
import math

from catboost import CatBoostRegressor

    
import pandas as pd

import sys
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime as dtdt
import seaborn as sns
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import GridSearchCV

In [248]:
product_id = '0021000615506'
shop_id = '013'

In [249]:
def get_TENPOURIAGE_TANPINNIKKEI_COMBINED_ALLDAY(product_id="",shop_id='',DEBUG=False):
    temppo_code = shop_id
    shouhin_code = product_id

    TENPOURIAGE_TANPINNIKKEI_COMBINED_ALLDAY = """
    WITH
        prodcut_id AS (VALUES ('{0}')),
        store_id AS (VALUES ('{1}'))

    select to_date(売上日,'YYYYMMDD') as 売上日 , 売上点数
        from 店舗売上単品日計_201112

    where 店舗コード=(table store_id)
    and 商品コード=(table prodcut_id)

    UNION ALL

    select to_date(売上日,'YYYYMMDD') as 売上日 , 売上点数
        from 店舗売上単品日計_200710_dtaka

    where 店舗コード=(table store_id)
    and 商品コード=(table prodcut_id)
    and 売上日 < (select 売上日 from 店舗売上単品日計_201112 where 店舗コード=(table store_id) and 商品コード=(table prodcut_id) order by 売上日 asc limit 1)


    order by 売上日 asc , 売上点数

    """.format(shouhin_code,temppo_code)

    if DEBUG == True:
            print ("DEBUG # product sale count from multiple table :",TENPOURIAGE_TANPINNIKKEI_COMBINED_ALLDAY)

    return TENPOURIAGE_TANPINNIKKEI_COMBINED_ALLDAY


In [250]:
# print(get_TENPOURIAGE_TANPINNIKKEI_COMBINED_ALLDAY(product_id="4901002148935",shop_id='013'))

In [251]:
import sys
import psycopg2
import pandas as pd


product_list =[]
DEBUG = True




def connectProductSale(product_list  = None,_path=None,shop_id = ''):
    mydataframe = pd.DataFrame()
    for product_id in product_list:
        conn = None
 

        try:
            if DEBUG ==True:
                print('Connecting to PostgreSql database...')


            conn = psycopg2.connect(host='172.16.3.28' ,dbname="920088", user="kenpin", password="kenpin")
            
            
            cur = conn.cursor()


            if DEBUG ==True:
                print('PostgreSQL database vestion:')


            cur.execute(get_TENPOURIAGE_TANPINNIKKEI_COMBINED_ALLDAY(product_id,shop_id))
            data = cur.fetchone()

            strline = ""
            while data is not None:
                
                new_row = {'date':data[0],'count':data[1]}
                mydataframe = mydataframe.append(new_row, ignore_index=True)
                
                data = cur.fetchone()

            cur.close()
            writefile.close()


        except (Exception ,psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()

                print('DB CLOSED')
                
        return mydataframe

In [252]:
dd = connectProductSale(product_list  = [product_id],_path=None,shop_id = shop_id)

Connecting to PostgreSql database...
PostgreSQL database vestion:
name 'writefile' is not defined
DB CLOSED


In [253]:


df = dd.copy()

In [254]:
dd = dd.set_index('date')

In [255]:
dd.index 

Index([2016-04-29, 2016-05-01, 2016-05-04, 2016-05-06, 2016-05-12, 2016-05-15,
       2016-05-16, 2016-05-18, 2016-05-19, 2016-05-21,
       ...
       2020-10-06, 2020-10-10, 2020-10-11, 2020-10-17, 2020-10-18, 2020-10-19,
       2020-10-20, 2020-10-24, 2020-10-29, 2020-11-11],
      dtype='object', name='date', length=532)

In [256]:
df [ 'date'] =pd.to_datetime(df['date'])

In [257]:
df2 = df[df['date'] >='2017-05-01']

In [258]:
df2=df2[df2['date']<='2020-08-01']

In [259]:
df2

,count,date
115,1,2017-05-03
116,1,2017-05-05
117,1,2017-05-06
118,1,2017-05-11
119,1,2017-05-13
...,...,...
493,1,2020-07-27
494,1,2020-07-28
495,1,2020-07-29
496,1,2020-07-30


In [260]:
datearray

DatetimeIndex(['2017-05-01', '2019-06-30', '2019-07-01', '2019-07-31',
               '2019-08-01', '2019-09-01', '2019-09-02', '2019-09-30',
               '2019-10-01', '2020-01-06', '2020-01-31', '2020-02-01',
               '2020-03-01', '2020-04-01', '2020-04-01', '2020-05-01',
               '2020-05-01', '2020-05-28', '2020-06-01', '2020-06-01',
               '2020-07-01', '2020-07-01', '2020-08-01', '2020-08-01'],
              dtype='datetime64[ns]', freq=None)

# SPECIAL SALE READ DATABASE

In [278]:
def specialSale(pid,sid):
    
    SPECIAL_SALE_Q = """

    WITH
        prodcut_id AS (VALUES ('{0}')),
        store_id AS (VALUES ('{1}'))

    select  
           --ts.商品コード,
           to_date(tk.販売開始日,'YYYYMMDD') as 販売開始日,
           to_date(tk.販売終了日,'YYYYMMDD') as 販売終了日

            --,abs(DATE_PART('day', to_date(tk.納品終了日,'YYYYMMDD')::timestamp - to_date(tk.納品開始日,'YYYYMMDD')::timestamp)) as days
           from 特売商品 ts,特売企画 tk
           where ts.店舗コード = (table store_id)
           and ts.商品コード in (select 商品コード	from 発注勧告商品補助	where 店舗コード = (table store_id) and 削除フラグ='0'	and 自動発注区分='2' )
           and ts.店舗コード = tk.店舗コード
           and ts.特売企画コード = tk.特売企画コード
           and tk.削除フラグ='0'
           and tk.ＥＯＳ区分='1'
           and ts.削除フラグ='0'
           and ts.停止フラグ='0'

           and 
	       abs(DATE_PART('day', to_date(tk.販売終了日,'YYYYMMDD')::timestamp - to_date(tk.販売開始日,'YYYYMMDD')::timestamp))<=7           and 
            (tk.特売種別<>'1' and tk.特売種別<>'2')


           and 商品コード in ((table prodcut_id))
           order by  ts.商品コード,tk.販売開始日

    """.format(pid,sid)

    return SPECIAL_SALE_Q

In [279]:
import sys
import psycopg2
import pandas as pd


product_list =[]
DEBUG = True




def connectSpecialSale(product_list  = None,_path=None,shop_id = ''):
    mydataframe = pd.DataFrame()
    for product_id in product_list:
        conn = None
 

        try:
            if DEBUG ==True:
                print('Connecting to PostgreSql database...')


            conn = psycopg2.connect(host='172.16.3.28' ,dbname="920088", user="kenpin", password="kenpin")
            
            
            cur = conn.cursor()


            if DEBUG ==True:
                print('PostgreSQL database vestion:')


            cur.execute(specialSale(pid=product_id,sid=shop_id))
            data = cur.fetchone()

            strline = ""
            while data is not None:
                
                new_row = {'datestart':data[0],'dateend':data[1]}
                mydataframe = mydataframe.append(new_row, ignore_index=True)
                
                data = cur.fetchone()

            cur.close()


        except (Exception ,psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()

                print('DB CLOSED')
                
        return mydataframe

In [280]:
dd2 = connectSpecialSale(product_list  = [product_id],_path=None,shop_id = shop_id)

Connecting to PostgreSql database...
PostgreSQL database vestion:
DB CLOSED


In [281]:
dd2

,dateend,datestart
0,2019-07-21,2019-07-15
1,2019-08-04,2019-07-29
2,2019-08-18,2019-08-12
3,2019-09-01,2019-08-26
4,2019-09-08,2019-09-02
5,2019-09-22,2019-09-16
6,2019-10-06,2019-09-30
7,2019-10-20,2019-10-14
8,2019-10-27,2019-10-21
9,2019-11-03,2019-10-28


## make special sale start to end date all 1 ,and other dates to 0

In [282]:
special_datelist = set()
for index,row in dd2.iterrows():
    #print(row['datestart'],row['dateend'])
    datelist = pd.date_range(start=row['datestart'], end=row['dateend'])
    
    
    for a in datelist:
        special_datelist.add(str(a.date()))
    #print(temp)
    # date list from start sale date to end sale date
    # will sate to 1 for each
    #special_datelist.extend(temp)
    
    #print(datelist)

In [283]:
# special_datelist

In [284]:


datelist = pd.date_range(start='2017/5/1', end='2020/9/1')

dataarray = []


for a in datelist:
    if str(a.date()) in special_datelist:
#         print(1)
        dataarray.append({'date': str(a.date()), 'spflag': 1})
    else:
        
        dataarray.append({'date': str(a.date()), 'spflag': 0})


# print(dataarray)

In [285]:
df_sp = pd.DataFrame(dataarray)

In [286]:
df_sp['date'] = pd.to_datetime(df_sp['date'])

In [287]:
df_sp

,date,spflag
0,2017-05-01,0
1,2017-05-02,0
2,2017-05-03,0
3,2017-05-04,0
4,2017-05-05,0
...,...,...
1215,2020-08-28,0
1216,2020-08-29,0
1217,2020-08-30,0
1218,2020-08-31,0


In [288]:
 tograph = pd.merge(df_sp,df2,how='left',left_on=['date'],right_on=['date']) 

In [289]:
tograph

,date,spflag,count
0,2017-05-01,0,NaN
1,2017-05-02,0,NaN
2,2017-05-03,0,1
3,2017-05-04,0,NaN
4,2017-05-05,0,1
...,...,...,...
1215,2020-08-28,0,NaN
1216,2020-08-29,0,NaN
1217,2020-08-30,0,NaN
1218,2020-08-31,0,NaN


In [290]:
tograph['count'] =tograph['count'].fillna(0)

In [291]:
df_sp[df_sp['spflag']>0] 

,date,spflag
805,2019-07-15,1
806,2019-07-16,1
807,2019-07-17,1
808,2019-07-18,1
809,2019-07-19,1
...,...,...
1192,2020-08-05,1
1193,2020-08-06,1
1194,2020-08-07,1
1195,2020-08-08,1


In [292]:
x = tograph['date']
y= tograph['spflag']
y2 = tograph['count']

In [293]:
%matplotlib qt

In [294]:
plt.figure()

plt.plot(x, y)
plt.plot(x, y2)
plt.title('linear')
plt.grid(True)